In [ ]:
# default_exp core

# RNAseq Pipelines using Dask and Prefect

> A framework for processing RNAseq data using Dask/Prefect.

In [ ]:
#hide
from nbdev.showdoc import *

# Practicing nbdev

In [ ]:
#export
def practice(txt: str)-> str:
    return(f'The text is {txt}')

In [ ]:
practice('nbdev')

'The text is nbdev'

In [ ]:
assert practice('nbdev')=="The text is nbdev"